In [27]:
import numpy as np
import sys, os
sys.path.append(os.curdir)
from common.util import im2col
from common.layers import *
from collections import OrderedDict
from common.trainer import Trainer
from dataset.mnist import load_mnist

In [2]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [3]:
x[0,0]

array([[0.48018931, 0.98944359, 0.79713185, 0.43279177, 0.2926893 ,
        0.74625363, 0.02342966, 0.45838274, 0.8364232 , 0.82140751,
        0.36687044, 0.49682652, 0.47797984, 0.2754657 , 0.48602463,
        0.26274569, 0.31477894, 0.96435953, 0.00923326, 0.96483567,
        0.7444617 , 0.1128689 , 0.83711175, 0.87601369, 0.03478435,
        0.27976502, 0.94103109, 0.80126365],
       [0.3807866 , 0.56587503, 0.36713318, 0.22787165, 0.20161384,
        0.20288219, 0.47129784, 0.54103465, 0.18075023, 0.07925868,
        0.29929825, 0.34132044, 0.63784486, 0.5272697 , 0.84890412,
        0.31548371, 0.71296176, 0.58548351, 0.85720447, 0.11994491,
        0.65126206, 0.90976559, 0.76847361, 0.65434655, 0.55841311,
        0.55863937, 0.88395918, 0.47850279],
       [0.21867643, 0.6621717 , 0.28274931, 0.65340225, 0.44440594,
        0.824372  , 0.23444838, 0.3830155 , 0.24150009, 0.18875102,
        0.5108781 , 0.15217609, 0.74227727, 0.64216052, 0.6963592 ,
        0.71776031, 0.8527

In [4]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

##9는 필터가 총 3 * 3 돌아가면 연산해야 되서
##75는 출력층 크기 한번 입력하는크기 25 * 채널 개수 3

(9, 75)


In [5]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [8]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FN, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1)
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        out = np.max(col, axis=1)
        
        out = out.resahep(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

In [ ]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={"filter_num": 30, "filter_size":5, "pad":0, "stride":1},
                hidden_size = 100, output_size = 10, weight_init_std = 0.01):
        filter_num = conv_param["filter_num"]
        filter_size = conv_param["filter_size"]
        filter_pad = conv_param["pad"]
        filter_stride = conv_param["stride"]
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params={}
        self.params["W1"] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params["b1"] = np.zeros(filter_num)
        self.params["W2"] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params["b2"] = np.zeros(hidden_size)
        self.params["W3"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b3"] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads={}
        grads["W1"] = self.layers["Conv1"].dW
        grads["b1"] = self.layers["Conv1"].db
        grads["W2"] = self.layers["Affine1"].dW
        grads["b2"] = self.layers["Affine1"].db
        grads["W3"] = self.layers["Affine2"].dW
        grads["b3"] = self.layers["Affine2"].db

        return grads
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]
    
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [43]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

In [ ]:
network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
max_epochs = 20

trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.300514874940209
=== epoch:1, train acc:0.315, test acc:0.275 ===
train loss:2.2985357459494677
train loss:2.2946133683149394
train loss:2.28842089833306
train loss:2.2863767030861757
train loss:2.2776113580826007
train loss:2.2660324589305003
train loss:2.2510867746533147
train loss:2.233880501998746
train loss:2.2129009356443694
train loss:2.1770925077443506
train loss:2.1361409630124175
train loss:2.097056447109827
train loss:2.038012981888908
train loss:1.9922204657351512
train loss:1.9805589607859277
train loss:1.9163486249167394
train loss:1.8601323505360277
train loss:1.6855142859893621
train loss:1.6545692331783215
train loss:1.5313424066339882
train loss:1.553077746129827
train loss:1.5008704374808637
train loss:1.3068338442157097
train loss:1.3127709483016017
train loss:1.047637912832977
train loss:1.1457413225718407
train loss:1.088157508818762
train loss:1.0094322682683765
train loss:0.9716542309307173
train loss:0.9619628130776633
train loss:0.8520473222943851


train loss:0.21331409020479392
train loss:0.25730313678056355
train loss:0.23183868668255417
train loss:0.05203874020830595
train loss:0.19516647481727428
train loss:0.18265346562764215
train loss:0.2174648011917158
train loss:0.26262885755906606
train loss:0.18819520839270276
train loss:0.11772183215511477
train loss:0.297384953360536
train loss:0.26986537703500696
train loss:0.16078332503755394
train loss:0.22861469207620352
train loss:0.10889865284067217
train loss:0.12244252845874883
train loss:0.22969017446970774
train loss:0.1828324681807207
train loss:0.26609567160642805
train loss:0.23644425612645734
train loss:0.22893159072642327
train loss:0.16554559606142621
train loss:0.24697523032154842
train loss:0.2237044665402837
train loss:0.18049111120427244
train loss:0.5240218005273822
train loss:0.12424462173569642
train loss:0.22547453344581458
train loss:0.14610913288841668
train loss:0.19594202791522933
train loss:0.15724099668085767
train loss:0.20691095985257618
train loss:0.0

train loss:0.05647049281929684
train loss:0.13669923759123984
train loss:0.057250949447735025
train loss:0.24329279441874035
train loss:0.14473343433029384
train loss:0.17111447943588337
train loss:0.11239639814546805
train loss:0.07173291848005962
train loss:0.06315385205755117
train loss:0.17247723223151304
train loss:0.1958326755683063
train loss:0.19947460178024962
train loss:0.09624691236506923
train loss:0.11246210841611357
train loss:0.12164670053590275
train loss:0.143043749909074
train loss:0.1597660707526719
train loss:0.06772155052956054
train loss:0.10458253850389228
train loss:0.07974924172151811
train loss:0.07308188889242966
train loss:0.12981697193015748
train loss:0.10171569756771953
train loss:0.06560326496685463
train loss:0.0789652139329697
train loss:0.12577677017824418
train loss:0.1378308290944626
train loss:0.07635389844803249
train loss:0.07938174066941726
train loss:0.14311086791852093
train loss:0.11196353601760006
train loss:0.047691573298523214
train loss:0

train loss:0.045371433305060105
train loss:0.030795357181703946
train loss:0.08906979893481144
train loss:0.03437661409329747
train loss:0.09802570269913921
train loss:0.08182510219360901
train loss:0.039183329917560884
train loss:0.11958881314348675
train loss:0.14539075838320706
train loss:0.05360765866771968
train loss:0.07672083111524111
train loss:0.08357527547217353
train loss:0.25115922972615523
train loss:0.04136376522104131
train loss:0.11444492523833207
train loss:0.1121527178837702
train loss:0.0514937297248908
train loss:0.06038241608407273
train loss:0.10091267339885096
train loss:0.05012652948576567
train loss:0.05577826561036009
train loss:0.10449880730939214
train loss:0.19298336606881095
train loss:0.09581614652796624
train loss:0.058101793823541865
train loss:0.070050905953013
train loss:0.10766218201541805
train loss:0.06819977648726787
train loss:0.030909962656928344
train loss:0.051567954857945256
train loss:0.08847714595402627
train loss:0.13821714748444486
train 

train loss:0.05508924858077493
train loss:0.08226039003034906
train loss:0.03582964776210234
train loss:0.03171875594816587
train loss:0.05906970463752553
train loss:0.03081987951145238
train loss:0.10447186307181397
train loss:0.061101180164243235
train loss:0.17875832008982534
train loss:0.0625432064379893
train loss:0.0647619355048195
train loss:0.04623689847108868
train loss:0.04376805770967862
train loss:0.05145528616832966
train loss:0.07022375913252069
train loss:0.06775581168697711
train loss:0.04021126421508759
train loss:0.06134105859227005
train loss:0.043618471047883335
train loss:0.052844728076142274
train loss:0.019378534382785865
train loss:0.02396780181622756
train loss:0.054644790375428035
train loss:0.10928998320253307
train loss:0.06511219793402286
train loss:0.05098802543913993
train loss:0.03832157291318533
train loss:0.10014852322873762
train loss:0.14154547392448458
train loss:0.04207006140496685
train loss:0.02437694729941432
train loss:0.058530132526488705
trai

...

train loss:0.0012267098764581334
train loss:0.001053391253512907
train loss:0.002520765299358011
train loss:0.0034479021357121226
train loss:0.001395652961127819
train loss:0.00045961834474815896
train loss:0.0008319076573844706
train loss:0.0017805220582330352
train loss:0.016537014242059653
train loss:0.001446030115738756
train loss:0.0011910423339410513
train loss:0.1810207195495896
train loss:0.001551853207920739
train loss:0.004795512818407581
train loss:0.00470101774689349
train loss:0.0021727742830828147
train loss:0.016691954414778467
train loss:0.010118615473374342
train loss:0.0027838155288609358
train loss:0.0006070266743877164
train loss:0.0013962163965402469
train loss:0.001997025900115159
train loss:0.0018625690621749736
train loss:0.0005866128039962326
train loss:0.005385754611504563
train loss:0.0061798345740848495
train loss:0.002618351787373386
train loss:0.0025099571816148774
train loss:0.011052563385966578
train loss:0.010247627930024513
train loss:0.000952522241659

train loss:0.017159051664627528
train loss:0.0013302620135844467
train loss:0.006566143372585159
train loss:0.0009272276502566637
train loss:0.0050689647917617296
train loss:0.0028064331187114184
train loss:0.0019030435413880198
train loss:0.0003385376385013065
train loss:0.011554477824225952
train loss:0.004689793937982349
train loss:0.0033516054024464387
train loss:0.0018991974392306554
train loss:0.010952939941866276
train loss:0.002243021063404073
train loss:0.005750828989640111
train loss:0.007263224225793778
train loss:0.00517981962769119
train loss:0.0038772250871182633
train loss:0.005334804396677542
train loss:0.005680920914748267
train loss:0.0005523734514954466
train loss:0.0035253369978699256
train loss:0.0019183800576015864
train loss:0.007729374294890049
train loss:0.008145731942270003
train loss:0.0014151524049473936
train loss:0.0015082768988139367
train loss:0.003158663324087826
train loss:0.009918956303383415
train loss:0.01875693039321266
train loss:0.016276997390778

train loss:0.01647691841062275
train loss:0.003645378364348234
train loss:0.002023768923260602
train loss:0.0025862952804491606
train loss:0.0065451854923090115
train loss:0.000971305349812069
train loss:0.0008682200855209384
train loss:0.0046900209150735755
train loss:0.001502167807891226
train loss:0.003096316673679118
train loss:0.00402680888607085
train loss:0.000749190188663584
train loss:0.003814337893131399
train loss:0.008998495431616923
train loss:0.00099547015617427
train loss:0.009495712572472469
train loss:0.0017677886715077532
train loss:0.008819548275066525
train loss:0.0005356062845277136
train loss:0.0009109207624474026
train loss:0.0022531097151761354
train loss:0.0011229572958610663
train loss:0.00479397278254418
train loss:0.006365381124238756
train loss:0.0021496169944172664
train loss:0.002150683410788451
train loss:0.0009163593935303263
train loss:0.0005416484202741812
train loss:0.0017739252483171077
train loss:0.0012312217467005896
train loss:0.00015708244895380

train loss:0.0016420591652638473
train loss:0.0023470974056593896
train loss:0.0009478030710905392
train loss:0.004420571040314584
train loss:0.00531511940513279
train loss:0.026448759266760283
train loss:0.001187419786788638
train loss:0.009248347297313967
train loss:0.00020152484876769148
train loss:0.0006017365238253032
train loss:0.0016598413670424303
train loss:0.007091956824785512
train loss:0.001497696279218882
train loss:0.0015338588730676181
train loss:0.0004817051645504261
train loss:0.0001520651471969562
train loss:0.00045253434692573514
train loss:0.0028026891062740904
train loss:0.002913502015480964
train loss:0.006681356328508307
train loss:0.0010321047590006259
train loss:0.010293140864051484
train loss:0.001539230930738154
train loss:0.0016932282168877174
train loss:0.003984120619106174
train loss:0.0011168978841288276
train loss:0.0018548672786630157
train loss:0.0006189527315262162
train loss:0.01687411512864979
train loss:0.0015493936773715352
train loss:0.0014427924

train loss:0.004505031962570154
train loss:0.0017557317693475676
train loss:0.0009531861823617436
train loss:0.003004420961791634
train loss:0.001822362866884514
train loss:0.0044206303474077675
train loss:0.05825920015830856
train loss:0.0008282687874277544
train loss:0.0025366812508557505
train loss:0.009841008484636505
train loss:0.007517886746644651
train loss:0.0027028955402046756
train loss:0.00048612175418872217
train loss:6.443807034347518e-05
train loss:0.0010624816899196656
train loss:8.892240640589856e-05
train loss:0.004004890310414252
train loss:0.0003384302052709757
train loss:0.007639414480287193
train loss:0.015269112737001732
train loss:0.0007763667057701945
train loss:0.0023470058163750016
train loss:0.008384085091681471
train loss:0.0018233464919008907
train loss:0.001105389782925
train loss:0.004625287623884088
train loss:0.003728114168846094
train loss:0.001133602987036912
train loss:0.0012426164444172714
train loss:0.0016780230413030528
train loss:0.00177133436256

In [ ]:
def filter_show(filters, nx=8, margin=3, scale=10):
    FN, C, FH, FW = filters.shape
    ny = int(np.ceil(FN / nx))

    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(FN):
        ax = fig.add_subplot(ny, nx, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i, 0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()
    
filter_show(network.params['W1'])